---
# Setting
---

<br>

## Import Modules

In [1]:
import gc
gc.collect()

0

In [2]:
# !pip install pytimekr
# !pip install optuna
# !pip install datatable

In [3]:
# d MyPython/2_Dacon_JEJU/DAT/
# unzip open.zip

In [4]:
import matplotlib as mpl
import matplotlib.font_manager as fm

fe = fm.FontEntry(fname='../NanumFont/NanumGothic.ttf',name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)  # or append is fine
mpl.rcParams['font.family'] = fe.name # = 'your custom ttf font name'

In [58]:
import os
import sys
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

import numpy as np
from tqdm import tqdm, trange
tqdm.pandas()

import warnings
warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
os.environ['PYTHONWARNINGS']='ignore::FutureWarning'

import itertools
import datetime
from pytimekr import pytimekr
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing as mp
from joblib import Parallel, delayed

import datatable as dt

In [6]:
# https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution

import contextlib
import joblib
from tqdm import tqdm

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [7]:
def abline(slope, intercept, color):
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '--',color=color)
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
        
def cnt(x):
    vc = x.value_counts().sort_index()
    res = pd.DataFrame({
        'index' : vc.index,
        'freq'  : vc.values,
    })
    res['rate'] = 100 * res['freq'] / res['freq'].sum()
    return res

In [8]:
from sklearn.metrics import mean_absolute_error

# verbose=0로 만들어주는 함수
# (참조) https://stackoverflow.com/questions/11130156/suppress-stdout-stderr-print-from-python-functions
class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

<br>

User Functions

In [50]:
module_path = "/home/ec2-user/SageMaker/Temp/Dacon/5_제주도도로교통량예측/lib"
if module_path not in sys.path:
    sys.path.append(module_path)

In [51]:
from send_email import send_email

<br>

## Initial Values

In [10]:
DAT_PATH = "../DAT/"

start_time = datetime.datetime.now()
print(start_time)

2022-11-12 18:12:50.236427


In [11]:
DL_SETTING = '/device:GPU:0' #'/device:CPU:0'

<br></br>

---
# Modeling
---
- tf.data.dataset 참조
    - [참조1](https://nodoudt.tistory.com/43)
    - [참조2](https://ericabae.medium.com/tensorflow-2-0-csv-%ED%8C%8C%EC%9D%BC-%ED%98%95%EC%8B%9D-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0-eddaa88d3112)

<br>

## DL

In [12]:
import tensorflow as tf
# import tensorflow.compat.v1 as tf
from tensorflow.keras.layers import Input, Dense, RepeatVector, LSTM, GRU, TimeDistributed, Bidirectional, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.constraints import NonNeg
from tensorflow.keras.optimizers import Adam

# Hyperparameter Optimization
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

In [13]:
# loss function for DL
def huber_loss(true, pred):
    threshold = 1
    error = true - pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [14]:
def get_dataset(file_path,batch_size,num_epochs,is_pack,shuffle,**kwargs):
    
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=batch_size,
        num_epochs=num_epochs,
        ignore_errors=True, 
        **kwargs
    )
    
    def pack(features, label):
        return tf.stack(list(features.values()), axis=-1), label
    
    if is_pack:
        dataset = dataset.map(pack)

    if shuffle:
        dataset = dataset.shuffle(500)
    
    return dataset

In [15]:
end_time = datetime.datetime.now()
run_time = end_time-start_time

print('='*45)
print(f'> start time : {start_time}')
print(f'>   end time : {end_time}')
print(f'>   run time : {run_time:}')
print('='*45)

> start time : 2022-11-12 18:12:50.236427
>   end time : 2022-11-12 18:13:00.942556
>   run time : 0:00:10.706129


<br></br>

### by all

In [ ]:
# X_train_oh = pd.read_parquet('../OUT/X_train_oh.parquet.gz')
# X_test_oh  = pd.read_parquet('../OUT/X_test_oh.parquet.gz')
# y_train    = pd.read_csv('../OUT/y_train.csv')

In [ ]:
# pd.concat([
#     X_train_oh.drop('segment',axis=1),
#     y_train,
# ],axis=1)\
#     .astype(float)\
#     .to_csv('../OUT/train_fn_oh_noseg.csv',index=False)
# X_test_oh.drop('segment',axis=1).astype(float).to_csv('../OUT/test_fn_oh_noseg.csv',index=False)

<br></br>

### by segment

## by segment

In [80]:
import glob
segment = [path.replace('../OUT/segment/','') for path in glob.glob('../OUT/segment/*')]
len(segment)

25

In [81]:
segment = pd.read_parquet(f'../OUT/X_train_oh.parquet.gz')['segment']

total_size = len(segment)
segment = segment.value_counts().sort_values(ascending=False).index

In [82]:
bg_check_df = pd.DataFrame(columns=['iter','total','segment','start_time','end_time','run_time'])

batch_size = 64
num_epochs = 1
epochs     = 256

In [85]:
%%time

with tf.device(DL_SETTING):

    total=len(segment)
    iter=0
    pbar = tqdm(segment)
    for seg in pbar:
        iter+=1
        
        iter_size = dt.fread(f'../OUT/segment/{seg}/train_df.csv').shape[0]
        each_epochs_size = int(np.ceil(iter_size / batch_size * num_epochs))
        text = f'{seg} : ModelSize={each_epochs_size:,}, DataSize={iter_size/total_size*100:.2f}%({iter_size:,}/{total_size:,})'
        pbar.set_description(text)

        start_time = datetime.datetime.now()
        
        train_dataset = get_dataset(
            file_path=f'../OUT/segment/{seg}/train_df.csv',
            batch_size=batch_size,
            num_epochs=num_epochs,
            label_name='target',
            is_pack=True,
            shuffle=True,
        )

        model = Sequential()
        model.add(Dense(units=64,activation='elu'))
        model.add(Dropout(0.2))
        model.add(Dense(units=32,activation='elu'))
        model.add(Dropout(0.2))
        model.add(Dense(units=1,activation=None,kernel_constraint='non_neg'))
        model.compile(optimizer=Adam(learning_rate=0.01),loss='huber',metrics=['mae'])

        # 'val_loss'
        mc = ModelCheckpoint(f'../MDL/DL/model_{seg}.h5', save_best_only=True, monitor='loss', mode='min')
        es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=30)

        model.fit(
            train_dataset,
            epochs=epochs,
            callbacks=[mc,es],
            verbose=1,
        )
        
        end_time = datetime.datetime.now()
        run_time = end_time-start_time
        
        tmp = pd.DataFrame({
            'iter' : iter,
            'total' : total,
            'segment' : seg,
            'start_time' : str(start_time),
            'end_time' : str(end_time),
            'run_time' : str(run_time),
        },index=[0])
        
        bg_check_df = pd.concat([bg_check_df,tmp],axis=0)
        bg_check_df.to_csv('../bg_check_df.csv',index=False)

In [86]:
send_email(
    to_emails='hjkim@storelink.io', 
    subject='Jeju Prediction Finished',
    email_body='Jeju Prediction Finished',
    attach_file='/home/ec2-user/SageMaker/Temp/Dacon/5_제주도도로교통량예측/bg_check_df.csv'
)

<br></br>

In [ ]:
# (참조) https://www.analyticsvidhya.com/blog/2020/10/multivariate-multi-step-time-series-forecasting-using-stacked-lstm-sequence-to-sequence-autoencoder-in-tensorflow-2-0-keras/
def CreateModel_DL(
    trial,
    X_tr,
    y_tr,
):
    
    if str(type(trial))=="<class 'optuna.trial._trial.Trial'>":
        setting_type = 'optuna'
    elif str(type(trial))=="<class 'dict'>":
        setting_type = 'dict'
    else:
        raise('unknown type of params')
        
    params = trial
    
    with tf.device(DL_SETTING):

        seed_everything(0)

        X_tr2, X_va2, X_te2, X_fc2 = X_tr.copy(), X_va.copy(), X_te.copy(), X_fc.copy()
        y_tr2, y_va2, y_te2, y_fc2 = y_tr.copy(), y_va.copy(), y_te.copy(), y_fc.copy()
        
        tr_index_info, va_index_info, te_index_info, fc_index_info = index_info

        #====================================================================================================#
        # (1) parameter setting
        #====================================================================================================#
        
        #----------------------------------------------------------------------------------------------------#
        # (1-1) model hyper-parameter
        #----------------------------------------------------------------------------------------------------#
        if setting_type=='optuna':
        
            n_layer = trial.suggest_int("n_layer", 1, 3)

            n_node = [np.nan]*n_layer
            n_node[0] = trial.suggest_int("n_node_1", 256, 1024) #64, 256)
            if n_layer>=2:
                for i in range(n_layer-1):
                    n_node[i+1] = trial.suggest_int(f"n_node_{i+2}", n_node[i]//2, n_node[i])

            dropout_rate = [np.nan]*n_layer
            for i in range(n_layer):
                dropout_rate[i] = trial.suggest_float(f"dropout_{i+1}", 0.0, 0.3)

            # https://hwk0702.github.io/ml/dl/deep%20learning/2020/07/09/activation_function/
            activation = trial.suggest_categorical('activation',["None","elu","relu"])

            is_bidirectional = trial.suggest_categorical('is_bidirectional',["True","False"])

            optimizer = trial.suggest_categorical('optimizer',["Adam","SGD","RMSprop"])

            algorithm = trial.suggest_categorical('algorithm',["LSTM","GRU"])

            is_timedistributed = trial.suggest_categorical('is_timedistributed',["True","False"])
        
        elif setting_type=='dict':
            
            # n_layer = 2
            # n_node = [128,64]
            # dropout_rate = [0.2,0.2]
            # activation = 'elu'
            # is_bidirectional = 'True'
            # algorithm = 'LSTM'
            # is_timedistributed = 'True'
            
            n_layer = params['n_layer']
            
            n_node = [np.nan]*n_layer
            for i in range(n_layer):
                n_node[i] = params[f"n_node_{i+1}"]
                    
            dropout_rate = [np.nan]*n_layer
            for i in range(n_layer):
                dropout_rate[i] = params[f"dropout_{i+1}"]
                
            activation = params['activation']
            
            is_bidirectional = params['is_bidirectional']

            optimizer = params['optimizer']

            algorithm = params['algorithm']

            is_timedistributed = params['is_timedistributed']
        
        else:
            raise('unknown type of params')
            
        #----------------------------------------------------------------------------------------------------#
        # (1-2) dataset 생성에 사용되는 parameter
        #----------------------------------------------------------------------------------------------------#
        if setting_type=='optuna':

            scale_x = trial.suggest_categorical('scale_x',["True","False"])
            scale_y = trial.suggest_categorical('scale_y',["True","False"])
            is_day_feature = trial.suggest_categorical('is_day_feature',["True","False"])
            is_feature_selection = trial.suggest_categorical('is_feature_selection',["True","False"])
        
        elif setting_type=='dict':
            
            scale_x = params['scale_x']
            scale_y = params['scale_y']
            is_day_feature = params['is_day_feature']
            is_feature_selection = params['is_feature_selection']
        
        else:
            raise('unknown type of params')
        
        dataset_params = {
            'scale_x': scale_x,
            'scale_y': scale_y,
            'is_day_feature' : is_day_feature,
            'is_feature_selection': is_feature_selection,
        }

        #----------------------------------------------------------------------------------------------------#
        # (1-3) get function from model hyper-parameter
        #----------------------------------------------------------------------------------------------------#
        # activation (last dense)
        if activation=='None':
            activation_x = None
        else:
            activation_x = activation

        # bidirectional
        if eval(is_bidirectional):
            bidirectional = Bidirectional
        else:
            bidirectional = identity

        # lstm / gru
        if algorithm=='LSTM':
            dl_alg = LSTM
        elif algorithm=='GRU':
            dl_alg = GRU

        # optimizer
        if optimizer == 'Adam':
            optimizer_x = tf.keras.optimizers.Adam()
        elif optimizer == 'SGD':
            optimizer_x = tf.keras.optimizers.SGD()
        elif optimizer == 'RMSprop':
            optimizer_x = tf.keras.optimizers.RMSprop()

        # timedistributed
        if eval(is_timedistributed):
            timedistributed = TimeDistributed
        else:
            timedistributed = identity

        #----------------------------------------------------------------------------------------------------#
        # (1-4) model 생성에 사용되는 parameter
        #----------------------------------------------------------------------------------------------------#
        model_params = {
            'n_layer' : n_layer,
            'activation' : activation,
            'is_bidirectional' : is_bidirectional,
            'optimizer' : optimizer,
            'algorithm' : algorithm,
            'is_timedistributed' : is_timedistributed,
        }
        for i in range(n_layer):
            model_params[f'n_node_{i+1}'] = n_node[i]
            model_params[f'dropout_{i+1}'] = dropout_rate[i]

        #====================================================================================================#
        # (2) scaling
        #====================================================================================================#
        scale_x = eval(dataset_params['scale_x'])
        scale_y = eval(dataset_params['scale_y'])
        scaler = None
        if (scale_x) | (scale_y):
            X_tr2, X_va2, X_te2, X_fc2, y_tr2, y_va2, y_te2, y_fc2, scalers = scaling(
                X_tr2,X_va2,X_te2,X_fc2,
                y_tr2,y_va2,y_te2,y_fc2,
                scale_x,scale_y,
            )
            if scale_y:
                scaler = scalers[f'scaler_{TARGET}']

        #====================================================================================================#
        # (3) day feature
        #====================================================================================================#
        if eval(dataset_params['is_day_feature']):

            # month
            for i in range(1,12+1):
                X_tr2[f'month_{i}'] = np.where(X_tr2.month==str(i),1,0)
                X_va2[f'month_{i}'] = np.where(X_va2.month==str(i),1,0)
                X_te2[f'month_{i}'] = np.where(X_te2.month==str(i),1,0)
                X_fc2[f'month_{i}'] = np.where(X_fc2.month==str(i),1,0)

            del X_tr2['month'], X_va2['month'], X_te2['month'], X_fc2['month']

            # day
            for i in range(1,31+1):
                X_tr2[f'day_{i}'] = np.where(X_tr2.day==str(i),1,0)
                X_va2[f'day_{i}'] = np.where(X_va2.day==str(i),1,0)
                X_te2[f'day_{i}'] = np.where(X_te2.day==str(i),1,0)
                X_fc2[f'day_{i}'] = np.where(X_fc2.day==str(i),1,0)

            del X_tr2['day'], X_va2['day'], X_te2['day'], X_fc2['day']

            # weekday
            for i in range(7):
                X_tr2[f'weekday_{i}'] = np.where(X_tr2.weekday==str(i),1,0)
                X_va2[f'weekday_{i}'] = np.where(X_va2.weekday==str(i),1,0)
                X_te2[f'weekday_{i}'] = np.where(X_te2.weekday==str(i),1,0)
                X_fc2[f'weekday_{i}'] = np.where(X_fc2.weekday==str(i),1,0)

            del X_tr2['weekday'], X_va2['weekday'], X_te2['weekday'], X_fc2['weekday']

        else:
            X_tr2 = X_tr2[setdiff(X_tr2.columns,['month','day','weekday','weekend','holiday'])]
            X_va2 = X_va2[setdiff(X_va2.columns,['month','day','weekday','weekend','holiday'])]
            X_te2 = X_te2[setdiff(X_te2.columns,['month','day','weekday','weekend','holiday'])]
            X_fc2 = X_fc2[setdiff(X_fc2.columns,['month','day','weekday','weekend','holiday'])]

        #====================================================================================================#
        # (4) feature selection
        #====================================================================================================#
        is_feature_selection = eval(dataset_params['is_feature_selection'])
        offset_info = None
        if is_feature_selection:
            X_tr2, X_va2, X_te2, X_fc2, offset_info = feature_selection(X_tr2, X_va2, X_te2, X_fc2, y_tr2)

        #====================================================================================================#
        # (5) make dataset
        #====================================================================================================#
        X_tr_list, y_tr_list = MakeDataset(tr_index_info, X_tr2, y_tr2, data_type='list')
        X_va_list, y_va_list = MakeDataset(va_index_info, X_va2, y_va2, data_type='list')
        X_te_list, y_te_list = MakeDataset(te_index_info, X_te2, y_te2, data_type='list')
        X_fc_list, y_fc_list = MakeDataset(fc_index_info, X_fc2, y_fc2, data_type='list')

        #====================================================================================================#
        # (6) Input shape
        #====================================================================================================#
        n_past     = X_tr_list.shape[1]
        n_future   = 1
        n_features = X_tr_list.shape[2]

        #====================================================================================================#
        # (7) modelling
        #====================================================================================================#

        #----------------------------------------------------------------------------------------------------#
        # encoder input
        #----------------------------------------------------------------------------------------------------#
        encoder_state  = [np.nan]*n_layer
        encoder_input  = [np.nan]*n_layer
        encoder_output = [np.nan]*n_layer
        for i_layer in range(n_layer):

            # return_sequences
            # : 마지막 layer에는 sequences를 return하면 안됨
            if i_layer==(n_layer-1):
                return_sequences = False
            else:
                return_sequences = True

            # dl algorithm input
            # : 처음에는 shape로, 이후에는 이전 encoder의 output을 사용
            if i_layer==0:
                encoder_input[i_layer] = Input(shape=(n_past, n_features))
            else:
                encoder_input[i_layer] = encoder_output[i_layer-1]

            # hidden layer
            dl_algorithm = bidirectional(dl_alg(
                units = n_node[i_layer], 
                dropout = dropout_rate[i_layer], 
                activation = activation_x,
                return_sequences = return_sequences, 
                return_state = True, 
            ))(encoder_input[i_layer])

            # states : LSTM을 각 타임 스텝별로 분리해서 넣어주는 경우가 있을 수 있다. 그럴 때는 initial_state를 사용
            # (참조) https://simpling.tistory.com/19
            encoder_output[i_layer], encoder_state[i_layer] = dl_algorithm[0], dl_algorithm[1:]

        #----------------------------------------------------------------------------------------------------#
        # decoder
        #----------------------------------------------------------------------------------------------------#
        decoder_output = [np.nan]*n_layer
        for i_layer in range(n_layer):

            # dl algorithm input
            # : 처음에는 shape로, 이후에는 이전 encoder의 output을 사용
            if i_layer==0:
                decoder_input = RepeatVector(n_future)(encoder_output[-1])
            else:
                decoder_input = encoder_output[i_layer-1]

            # hidden layer
            decoder_output[i_layer] = bidirectional(dl_alg(
                units = n_node[i_layer],
                dropout = dropout_rate[i_layer], 
                activation = activation_x,
                return_sequences = True,
            ))(decoder_input,initial_state = encoder_state[i_layer])

        #----------------------------------------------------------------------------------------------------#
        # TimeDistributed Dense
        #----------------------------------------------------------------------------------------------------#
        # output = timedistributed(Dense(1, activation = activation_x))(decoder_output[-1])
        output = timedistributed(Dense(1, activation = None, kernel_constraint='non_neg'))(decoder_output[-1])
        
        #====================================================================================================#
        # (8) Model Compile
        #====================================================================================================#
        # custom loss function by backend
        from tensorflow.keras.backend import constant
        from tensorflow.keras import backend as K

        def root_mean_squared_error(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true))) 

        def tf_std_weighted_scoring(y_true,y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true))) + (K.std(y_true)/(K.std(y_pred)+constant(1e-2)))**constant(2)
        
        model = tf.keras.models.Model(
            encoder_input[0],
            output,
        )
        model.compile(
            optimizer=optimizer_x, 
            loss=tf.keras.losses.Huber(), 
            # loss='mean_squared_error',
            # loss='mean_absolute_error',
            # loss='mean_absolute_percentage_error',
            # loss=tf_std_weighted_scoring,
            metrics=['mape'],
        )

        X = (X_tr_list, X_va_list, X_te_list, X_fc_list)
        y = (y_tr_list, y_va_list, y_te_list, y_fc_list)
        params = (dataset_params, model_params)
        info = (scaler, offset_info)

        return model, X, y, params, info

def FitModel_DL(
    trial,
    X_tr, X_va, X_te, X_fc,
    y_tr, y_va, y_te, y_fc,
    index_info,
    batch_size, 
    epochs,
    scoring,
    patience = None,
    verbose = 1,
    mc_file = DL_MC_PATH + 'dl.h5',
    seed = 0,
    slient = True,
    is_tqdm = True,
):
    
    if str(type(trial))=="<class 'optuna.trial._trial.Trial'>":
        setting_type = 'optuna'
    elif str(type(trial))=="<class 'dict'>":
        setting_type = 'dict'
    else:
        raise('unknown type of params')
        
    params = trial

    #global model, tr_data, va_data, te_data, fc_data, info
    with tf.device(DL_SETTING):

        start_time = time.time()

        X_tr2, X_va2, X_te2, X_fc2 = X_tr.copy(), X_va.copy(), X_te.copy(), X_fc.copy()
        y_tr2, y_va2, y_te2, y_fc2 = y_tr.copy(), y_va.copy(), y_te.copy(), y_fc.copy()
        
        tr_index_info, va_index_info, te_index_info, fc_index_info = index_info

        #----------------------------------------------------------------------------------------------------------------------#
        # (1) Clear clutter from previous session graphs.
        #----------------------------------------------------------------------------------------------------------------------#
        tf.keras.backend.clear_session()
        # seed_everything(seed=seed)
        # tf.random.set_seed(seed)

        #----------------------------------------------------------------------------------------------------------------------#
        # (2) with GPU
        #----------------------------------------------------------------------------------------------------------------------#
        strategy = tf.distribute.MirroredStrategy()
        with strategy.scope():

            #----------------------------------------------------------------------------------------------------------------------#
            # (3) model, dataset 생성
            #----------------------------------------------------------------------------------------------------------------------#
            model, X, y, params, info = CreateModel_DL(
                params,
                X_tr2, X_va2, X_te2, X_fc2,
                y_tr2, y_va2, y_te2, y_fc2,
                index_info,
            )

            X_tr_list, X_va_list, X_te_list, X_fc_list = X
            y_tr_list, y_va_list, y_te_list, y_fc_list = y
            dataset_params, model_params = params
            scaler, offset_info = info

            # Wrap data in Dataset objects.
            tr_data = tf.data.Dataset.from_tensor_slices((X_tr_list, y_tr_list))
            va_data = tf.data.Dataset.from_tensor_slices((X_va_list, y_va_list))
            te_data = tf.data.Dataset.from_tensor_slices((X_te_list, y_te_list))
            fc_data = tf.data.Dataset.from_tensor_slices((X_fc_list, y_fc_list))

            # The batch size must now be set on the Dataset objects.
            tr_data = tr_data.batch(batch_size)
            va_data = va_data.batch(batch_size)
            te_data = te_data.batch(batch_size)
            fc_data = fc_data.batch(batch_size)

            # Disable AutoShard.
            options = tf.data.Options()
            options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
            tr_data = tr_data.with_options(options)
            va_data = va_data.with_options(options)
            te_data = te_data.with_options(options)
            fc_data = fc_data.with_options(options)

            #----------------------------------------------------------------------------------------------------------------------#
            # (4) model callback 생성
            #----------------------------------------------------------------------------------------------------------------------#
            # Fit the model on the training data.
            # The TFKerasPruningCallback checks for pruning condition every epoch.

            # learning rate scheduler
            # scheduler = ExponentialDecay(1e-3, 400*((len(X_train_list)*0.8)/batch_size), 1e-5)
            # lr = LearningRateScheduler(scheduler, verbose=0)
            
            #lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
            #lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 0.05)
            
            # (참조) https://rinha7.github.io/keras-callbacks/
            #lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 0.2)
            #reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.2, patience=5, min_lr=0.001, min_delta=0.0001)
            mc = ModelCheckpoint(
                mc_file,              # file명을 지정합니다
                monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                verbose=0,            # 로그를 출력합니다
                save_best_only=True,  # 가장 best 값만 저장합니다
                mode='min'            # auto는 알아서 best를 찾습니다. min/max
            )
            if patience is None:
                callbacks = [mc]#,lr,reduce_lr]
            else:
                es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=patience)
                callbacks = [mc,es]#,lr,reduce_lr]

            #----------------------------------------------------------------------------------------------------------------------#
            # (5) model fitting
            #----------------------------------------------------------------------------------------------------------------------#
            # tr_data.batch(batch_size)에서 batch_size를 지정했기 때문에,
            # model.fit에서 batch_size를 지정하면 에러가 발생
            
            # if verbose==0:
            #     pbar = range(epochs)
            # else:
            #     pbar = tqdm(range(epochs))

            # loss_list = []
            # for i in pbar:
            #     seed_everything(0)
            #     model.fit(
            #         tr_data,
            #         #batch_size=batch_size,
            #         callbacks=callbacks,
            #         epochs=1,
            #         validation_data=va_data,
            #         verbose=verbose,
            #     )
            #     huber_loss_x = huber_loss(
            #         true = y_va_list.flatten(),
            #         pred = model.predict(va_data).flatten(),
            #     )
            #     loss_list.append(huber_loss_x)

            #     if patience is not None:
            #         es = Callback_EarlyStopping(loss_list,min_delta=None,patience=patience)
            #         if es:
            #             break

            seed_everything(0)
            model.fit(
                tr_data,
                #batch_size=batch_size,
                callbacks=callbacks,
                epochs=epochs,
                validation_data=va_data,
                verbose=verbose,
            )
            
            
            #---------------------------------------------------------------------------------------------------------#
            # (6) scoring
            #---------------------------------------------------------------------------------------------------------#
            va_pred = model.predict(va_data)
            va_true = y_va_list.copy()

            if eval(dataset_params['scale_y']):
                va_pred = scaler.inverse_transform(va_pred.reshape(-1,1))
                # va_pred = log_scaler_inv(va_pred, offset_info[TARGET_FEATURE])

                va_true = scaler.inverse_transform(va_true.reshape(-1,1))
                # va_true = log_scaler_inv(va_true, offset_info[TARGET_FEATURE])

            # reshape
            va_pred = va_pred.flatten()#.reshape(int(len(va_pred.flatten())/5),5,1).flatten()
            va_true = va_true.flatten()#.reshape(int(len(va_true.flatten())/5),5,1).flatten()

            score = scoring(true=va_true,pred=va_pred)
            pred_std = np.std(va_pred)
            true_std = np.std(va_true)
            data_range = max(va_true.flatten()) - min(va_true.flatten())

            end_time = time.time()
            run_time = end_time-start_time

            # print information
            global iter_dl
            iter_dl+=1
            
            # save best model
            dataset = (tr_data, va_data, te_data, fc_data)
            info = [scaler, offset_info]
            
            # trial.set_user_attr(key="best_model", value=model)
            # trial.set_user_attr(key="best_model_data", value=dataset)
            # trial.set_user_attr(key="best_model_info", value=info)

            # > 총 16가지
            # (1) scale_x : True, False
            # (2) scale_y : True, False
            # (3) is_day_feature : True, False
            # (4) is_feature_selection : True, False
            if not slient:
                print(f'{color.BOLD}{color.CYAN}({iter}/{n_trials}){color.END}')
                print(f'  > score={score:.3f}, pred_std={pred_std:.3f}, true_std={true_std:.3f}, range={data_range:.3f}, time={run_time:.2f}s')
                print(f'  > dataset_params:')
                for key,value in dataset_params.items():
                    print(f'      {key} : {value}')
                print(f'  > model_params :')
                for key,value in model_params.items():
                    print(f'      {key} : {value}')
                print('')
            else:
                if is_tqdm:
                    text = f'score={score:.3f}, pred_std={pred_std:.3f}, true_std={true_std:.3f}'
                    pbar_dl.set_description(text)
                    pbar_dl.update(1)
                else:
                    pass

            if setting_type=='optuna':
                return score

            elif setting_type=='dict':
                return model, dataset, info
            
            else:
                raise('unknown type of params')

In [ ]:
train_df7.to_parquet('../OUT/train_fn.parquet.gz')
test_df7 .to_parquet('../OUT/test_fn.parquet.gz')

<br></br>